In [ ]:
# First, we add all the packages we need to the system.
# This is what enables us to run all the code below.

import os
from dotenv import load_dotenv

from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.mcp import MCPStdioPlugin

In [ ]:
# Here, we connect to the Azure AI Foundry, and get the necessary environment variables
# including the project endpoint, model deployment name, and tenant ID.

load_dotenv()
endpoint = os.getenv("ENDPOINT")
model_deployment = os.getenv("MODEL_DEPLOYMENT_NAME")
api_key = os.getenv("API_KEY")
api_version = os.getenv("API_VERSION")

In [ ]:
# This block of code connects to the MCP Server
async with MCPStdioPlugin(
    name="InventoryManagementMCPPlugin",
    description="Inventory Management MCP Plugin",
    command="python",
    args=["server.py"],
) as inventory_plugin:
    pass
 
chat_service = AzureChatCompletion(
    endpoint=endpoint,
    deployment_name=model_deployment,
    api_version=api_version,
    api_key=api_key
)

In [ ]:
inventory_agent = ChatCompletionAgent(
    name="InventoryAgent",
    description="An agent that is knowledgeable about the inventory.",
    instructions="Handle research and information about the inventory, including stock levels and item details.",
    service=chat_service,
    plugins=[inventory_plugin],
)

await inventory_plugin.connect()

In [ ]:
# Asking the first question to the agent.
# The agent will use the MCP plugin to get the current product inventory levels.

TASK = "What are the current product inventory levels?"

print(f"😃: {TASK}")
response = await inventory_agent.get_response(messages=TASK)
print(f"🤖: {response.name}: {response}")


In [ ]:
# Asking the second question to the agent.
# The agent will use the MCP plugin to get the recommended products for clearance.

TASK = "Which products would you recommend for clearance?"

print(f"😃: {TASK}")
response = await inventory_agent.get_response(messages=TASK)
print(f"🤖: {response.name}: {response}")


In [ ]:
# Asking the third question to the agent.
# The agent will use the MCP plugin to get the best selling products this week.

TASK = "What are the best selling products this week?"

print(f"😃: {TASK}")
response = await inventory_agent.get_response(messages=TASK)
print(f"🤖: {response.name}: {response}")


In [ ]:
# Asking the fourth question to the agent.
# The agent will use the MCP plugin to get the recommended products to restock.

TASK = "Which products would you recommend to restock?"

print(f"😃: {TASK}")
response = await inventory_agent.get_response(messages=TASK)
print(f"🤖: {response.name}: {response}")


In [ ]:
# Asking the fifth question to the agent.
# The agent will use the MCP plugin to get product details.

TASK = "Can you tell me about Grandma's Boysenberry Spread?"

print(f"😃: {TASK}")
response = await inventory_agent.get_response(messages=TASK)
print(f"🤖: {response.name}: {response}")

In [ ]:
# Asking the sixth question to the agent.
# The agent will use the MCP plugin answer the users question.

# Users, come up with your own question here!
TASK = "Can..."

print(f"😃: {TASK}")
response = await inventory_agent.get_response(messages=TASK)
print(f"🤖: {response.name}: {response}")

In [ ]:
# Asking the last to the agent.
# The agent will use the MCP plugin to generate a marketing campaign for products that are under clearance.

TASK = "Can you generate a marketing campaign for products that are under clearance?"

print(f"😃: {TASK}")
response = await inventory_agent.get_response(messages=TASK)
print(f"🤖: {response.name}: {response}")